In this notebook, we will build a 3D map of a scene from a small set of images and refine it with the featuremetric optimization. We then localize an image downloaded from the Internet and show the effect of the refinement.

# Setup
We start by defining some output paths: where the intermediate files will be stored.

In [1]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
import os
import time
import sys
from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect
from PIL import Image, ImageDraw
import pycolmap
from pathlib import Path
import numpy as np
#import visualize_model
# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

In [2]:
import torch 
print(torch.__version__)
print(torch.cuda.get_arch_list())

1.9.1+cu111
['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86']


### Helper Functions

In [3]:
'''
colors the PLY file at the input path
with the provided custom color
output location => ply/output_ply_name
'''
def colorize_pointcloud(input_ply_path, color, output_path):
    ply_file_path = Path(input_ply_path)
        
    if ply_file_path.suffix != '.ply':
        raise ValueError(f"The file at {ply_file_path} is not a .PLY file.")
    
    
    model = pycolmap.Reconstruction()
    model.import_PLY(ply_file_path.as_posix())
    
    
    colors_dict = {
        "red": np.array([255, 0, 0]),
        "green": np.array([0, 255, 0]),
        "blue": np.array([0, 0, 255]),
        "yellow": np.array([255, 255, 0]),
        "cyan": np.array([0, 255, 255]),
        "magenta": np.array([255, 0, 255]),
        "white": np.array([255, 255, 255]),
        "black": np.array([0, 0, 0]),
        "orange": np.array([255, 165, 0]),
        "purple": np.array([128, 0, 128]),
        "brown": np.array([165, 42, 42]),
        "gray": np.array([128, 128, 128]),
        "pink": np.array([255, 192, 203]),
        "lime": np.array([0, 255, 0]),
        "indigo": np.array([75, 0, 130])
    }
    
    if color not in colors_dict:
        raise ValueError(f"Color {color} not found in color dictionary")

    
    custom_color = colors_dict[color]
    
    colored_model = pycolmap.Reconstruction()
    #for id, point in tqdm(model.points3D.items(), desc="Colorizing points"):
    for id, point in model.points3D.items():
        colored_model.add_point3D(xyz = point.xyz, track = pycolmap.Track(), color = custom_color)

    #output_ply_path =  "../ply/" + output_ply_name 
    #colored_model.export_PLY(output_ply_path.as_posix())
    colored_model.export_PLY(output_path)

In [4]:
import numpy as np

fx = 1093.2768
fy = 1093.2768
cx = 964.989
cy = 569.276

def get_camera_matrix(fx, fy, cx, cy):
    return np.array([[fx,0 , cx], [0, fy, cy], [0 , 0, 1]]).astype(np.float64)

def compute_cam_extrinsics(img):
    from read_write_model import qvec2rotmat
    R = qvec2rotmat(img.qvec)
    t = img.tvec.reshape(3,-1)
    R_t = np.concatenate((R,t), axis = 1)
    #R_t = np.vstack([np.array([0,0,0,1]), R_t])
    return R_t    # 3 * 4

def compute_projection_matrix(K, R_t):
    return np.dot(K, R_t)    


In [57]:
def transform_points_to_camera_frame(camera_rt_homo, points): 
    X = points
    ones = np.ones((X.shape[0], 1))
    X_homo = np.hstack((X, ones)) #homogeneous co-ordinates
    
    #camera_rt_homo = np.vstack((camera_rt, np.array([0, 0, 0, 1])))
    
    #print(f"camera_rt.shape: {camera_rt.shape} camera_rt_homo.shape: {camera_rt_homo.shape}")
    print(f"X.shape: {X.shape} X_homo.shape: {X_homo.shape}")
    
    Xc_homo = np.dot(camera_rt_homo ,X_homo.T).T #homogeneous co-ordinates in camera frame 
    #Xc_homo = np.dot(X_homo ,camera_rt_homo) #homogeneous co-ordinates in camera frame 
    
    print(f"Xc_homo.shape: {Xc_homo.shape}")
    
    #Xc_homo = np.dot(X_homo ,camera_rt_homo) #homogeneous co-ordinates in camera frame 
    Xc =  Xc_homo[:, :3] / Xc_homo[:, 3:] #dehomogenizing
    
    return Xc 

In [64]:
'''test cases for transform_points_to_camera_frame function ''' 
def test_identity_matrix():
    points = np.array([[1, 2, 3], [4, 5, 6]])  # replace with actual points
    camera_rt_homo = np.eye(4)  # identity matrix
    
    transformed_points = transform_points_to_camera_frame(camera_rt_homo,  points)
    print(f"transformed_points\n: {transformed_points}")
    #passed =  np.allclose(transform_points_to_camera_frame(camera_rt, points), points)
    #return ("pass" if passed is True else  "fail")
    
def test_custom_transformation_matrix():
    points = np.array([[1, 2, 3], [4, 5, 6]])  # points
    matrix = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, -1],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])  # custom transformation matrix
    #expected_result = np.array([[2, -1, 3], [6, -4, 6]])  # points after transformation
    #assert np.allclose(transform_points_to_camera_frame(points, matrix), expected_result)
    points_t = transform_points_to_camera_frame(matrix, points)
    print(f"points_t\n: {points_t}")


In [65]:
(test_custom_transformation_matrix())

X.shape: (2, 3) X_homo.shape: (2, 4)
Xc_homo.shape: (2, 4)
points_t
: [[1. 1. 3.]
 [4. 4. 6.]]


In [54]:
test_identity_matrix()

X.shape: (2, 3) X_homo.shape: (2, 4)
Xc_homo.shape: (2, 4)
transformed_points
: [[1. 2. 3.]
 [4. 5. 6.]]


In [21]:
'''
camera_rt => target camera extrinsics
model_input_path => path to the directory containing images.bin, points3d.bin, cameras.bin
'''
def transform_points3d_to_camera_frame(input_model, camera_rt):
    #from read_write_model import read_points3D_binary
    #sfm_points_path = model_input_path / "points3D.bin"
    #sfm_points_dict = read_points3D_binary(sfm_points_path)
    sfm_points_dict = input_model.points3D
    X = np.array([value.xyz for value in sfm_points_dict.values()]) #co-ordinates in world frame
    ones = np.ones((X.shape[0], 1))
    X_homo = np.hstack((X, ones)) #homogeneous co-ordinates
    
    camera_rt_homo = np.vstack((camera_rt, np.array([0, 0, 0, 1])))
    
    print(f"camera_rt.shape: {camera_rt.shape} camera_rt_homo.shape: {camera_rt_homo.shape}")
    print(f"X.shape: {X.shape} X_homo.shape: {X_homo.shape}")
    
    Xc_homo = np.dot(camera_rt_homo ,X_homo.T).T #homogeneous co-ordinates in camera frame 
    
    print(f"Xc_homo.shape: {Xc_homo.shape}")
    
    #Xc_homo = np.dot(X_homo ,camera_rt_homo) #homogeneous co-ordinates in camera frame 
    Xc =  Xc_homo[:, :3] / Xc_homo[:, 3:] #dehomogenizing
    
    return Xc

In [25]:
'''
transforms an input model to the frame of the target camera
'''
def transform_model_to_camera_frame(input_model:pycolmap.Reconstruction(), camera_rt) -> pycolmap.Reconstruction():
    transformed_points3d = transform_points3d_to_camera_frame(input_model, camera_rt)
    
    output_model = pycolmap.Reconstruction()
    counter = 0
    for (id, input_pt), new_pt in zip(input_model.points3D.items(), transformed_points3d):
        output_model.add_point3D(xyz = new_pt , track = pycolmap.Track(), color = input_pt.color)

    return output_model

In [7]:
#sfm_model_in_camera_frame.export_PLY("../ply/sfm_in_camera_frame.ply")

In [8]:
#colorizing sfm_in_camera_frame pointcloud
#
#colorize_pointcloud("../ply/sfm_in_camera_frame.ply", "orange", "sfm_in_camera_frame_orange.ply")

In [9]:
#colorizing svo pointcloud
#colorize_pointcloud("../svo_output/frame_20/pointcloud/pointcloud.ply", "yellow", "svo_yellow.ply")

#### PLY generation

In [10]:
sfm_path = Path("dense_reconstruction/output/")
sfm_model = pycolmap.Reconstruction()
sfm_model.read_binary(sfm_path.as_posix())

In [11]:
sfm_model.summary()

'Reconstruction:\n\tnum_reg_images = 80\n\tnum_cameras = 2\n\tnum_points3D = 2941007\n\tnum_observations = 52354\n\tmean_track_length = 0.0178014\n\tmean_observations_per_image = 654.425\n\tmean_reprojection_error = 0'

In [12]:
sys.path.append("/home/skumar/colmap/scripts/python")
from read_write_model import read_images_binary

In [13]:
# for image_id, image in sfm_model.images.items():
#     print(f"{image_id} {image.name}")

In [14]:
target_frame_id = 13 #left_20

In [15]:
left_img = sfm_model.images[13]
K = get_camera_matrix(fx,fy,cx,cy)
camera_rt = compute_cam_extrinsics(left_img)
#P = compute_projection_matrix(K, left_Rt)

In [16]:
camera_rt

array([[ 0.99572665, -0.09088052, -0.01640656,  0.02774167],
       [ 0.09212914,  0.98981585,  0.10852101,  0.01489788],
       [ 0.00637703, -0.10956878,  0.99395876, -0.07664639]])

In [31]:
Xc.shape

(2941007, 3)

In [28]:
model = pycolmap.Reconstruction()
type(model.points3D)

pycolmap.MapPoint3DIdPoint3D

In [66]:
sfm_model_in_camera_frame = transform_model_to_camera_frame(sfm_model, camera_rt)

camera_rt.shape: (3, 4) camera_rt_homo.shape: (4, 4)
X.shape: (2941007, 3) X_homo.shape: (2941007, 4)
Xc_homo.shape: (2941007, 4)


In [67]:
sfm_model_in_camera_frame.export_PLY("ply/sfm_in_camera_frame.ply")

In [68]:
sfm_model.export_PLY("ply/sfm.ply")

In [48]:
colorize_pointcloud("ply/sfm_in_camera_frame.ply", "orange", "ply/sfm_in_camera_frame_orange.ply")

In [70]:
colorize_pointcloud("ply/sfm.ply", "cyan", "ply/sfm_cyan.ply")